In [ ]:
!pip install diffusers transformers accelerate safetensors gradio rembg


: 

In [ ]:
from huggingface_hub import notebook_login
notebook_login()



In [ ]:
import torch
from diffusers import StableDiffusionPipeline

# Load the model (v1.5 example)
model_id = "runwayml/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,   # faster inference on GPU
    use_safetensors=True
)

# Move pipeline to GPU
pipe = pipe.to("cuda")


In [ ]:
import os
from PIL import Image

def generate_ad(prompt, out_dir="/content/drive/MyDrive/ad_outputs/", n=1, steps=20):
    os.makedirs(out_dir, exist_ok=True)
    paths = []
    for i in range(n):
        out = pipe(prompt, num_inference_steps=steps, guidance_scale=7.5)
        img = out.images[0]
        path = os.path.join(out_dir, f"ad_{i}.png")
        img.save(path)
        paths.append(path)
    return paths


In [ ]:
prompt = "Modern banner ad for a smartwatch sale, clean layout, white background, product focus"
files = generate_ad(prompt, n=3, steps=25)
files


In [ ]:
!pip install gradio
import gradio as gr

def gradio_generate(prompt):
    files = generate_ad(prompt, n=1)
    return files[0]

gr.Interface(fn=gradio_generate, inputs="text", outputs="image").launch()
